In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)


c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import sys
sys.path.append('../dataset')
import get_funsd
funsd = get_funsd.CustomFunsdDataset('../dataset/')
funsd.split_generators()
funsd

CustomFunsdDataset:
DatasetDict({
    train: Dataset({features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags','line_ids','linkings','image','image_name'], num_rows: 149}),
    test: Dataset({features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags','line_ids','linkings','image','image_name'], num_rows: 50})
})

In [5]:
label_map = {
    0: 'O',
    1: 'B-HEADER',
    2: 'I-HEADER',
    3: 'B-QUESTION',
    4: 'I-QUESTION',
    5: 'B-ANSWER', 
    6: 'I-ANSWER'}

In [6]:
from datasets import Dataset, DatasetDict

funsd_train_dataset = Dataset.from_dict({
    "id": [entry["id"] for entry in funsd["train"]],
    "tokens": [entry["tokens"] for entry in funsd["train"]],
    "ner_boxes": [entry["ner_boxes"] for entry in funsd["train"]],
    "bboxes": [entry["bboxes"] for entry in funsd["train"]],
    "ner_tags": [entry["ner_tags"] for entry in funsd["train"]],
    "line_ids": [entry["line_ids"] for entry in funsd["train"]],
    "linkings": [entry["linkings"] for entry in funsd["train"]],
    "image": [entry["image"] for entry in funsd["train"]],
    "image_name": [entry["image_name"] for entry in funsd["train"]],
})

funsd_test_dataset = Dataset.from_dict({
    "id": [entry["id"] for entry in funsd["test"]],
    "tokens": [entry["tokens"] for entry in funsd["test"]],
    "ner_boxes": [entry["ner_boxes"] for entry in funsd["test"]],
    "bboxes": [entry["bboxes"] for entry in funsd["test"]],
    "ner_tags": [entry["ner_tags"] for entry in funsd["test"]],
    "line_ids": [entry["line_ids"] for entry in funsd["test"]],
    "linkings": [entry["linkings"] for entry in funsd["test"]],
    "image": [entry["image"] for entry in funsd["test"]],
    "image_name": [entry["image_name"] for entry in funsd["test"]],
})

# Optionally, you can create a DatasetDict if you have train/test splits
dataset = DatasetDict({
    "train": funsd_train_dataset,
    'test': funsd_test_dataset
})


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags', 'line_ids', 'linkings', 'image', 'image_name'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags', 'line_ids', 'linkings', 'image', 'image_name'],
        num_rows: 50
    })
})

In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
import sys
sys.path.append('../..')
from mytools import unnormalize_box, normalize_box

In [10]:

import numpy as np
def make_dataset(documents):
    images = documents['image']
    
    rgb_images = list(map(lambda img: img.convert('RGB'), images))
  
    words = documents['tokens']
    
    # boxes = documents['bboxes'] ## token boxes normalized below
    
    boxes = documents['ner_boxes']
    
    word_labels = documents['ner_tags']
    
    normalized_boxes = []

    # Normalize the boxes by image width and height
    for i in range(len(boxes)):
        normalized_boxes.append([])
        for j in range(len(boxes[i])):
            normalized_boxes[-1].append(normalize_box(boxes[i][j], images[i].width, images[i].height))

    encoding = processor(rgb_images, words, boxes=normalized_boxes, word_labels=word_labels,
                         truncation=True, padding="max_length", return_offsets_mapping=True)
    
    encoding.pop('offset_mapping')

    return encoding

In [11]:
cols = funsd_train_dataset.column_names

In [12]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})
train_dataset = dataset["train"].map(
    make_dataset,
    batched=True,
    remove_columns = cols,
    features=features)

test_dataset = dataset["test"].map(
    make_dataset,
    batched=True,
    remove_columns = cols,
    features=features)


Map: 100%|██████████| 50/50 [00:00<00:00, 79.44 examples/s]


In [13]:
train_dataset.set_format("torch")
test_dataset.set_format("torch")

In [14]:
from evaluate import load 
metric = load("seqeval")

In [15]:
label_list = ['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER']


In [16]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [17]:
label_map_reversed = {v: k for k, v in label_map.items()} 

In [18]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.data.data_collator import default_data_collator
from transformers import LayoutLMv3ForTokenClassification
from transformers.trainer_callback import TrainerCallback


In [1]:
import pickle, json
with open("optuna_study.pkl", "rb") as f:
    study = pickle.load(f)

with open("tuning.json", "r") as f:
    hyperparameter_dict = json.load(f)

c:\Users\PX\Master\διπλωμαιτκ\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
study.best_params

{'dropout': 0.3560592494053517,
 'learning_rate': 3.127610016621582e-05,
 'weight_decay': 0.000817139913231531}

In [4]:
import numpy as np
def find_best_epochs_metrics(hyperparameter_dict):
    best_epochs_data = {}
    
    for trial_num, trial_data in hyperparameter_dict.items():
        metrics_history = trial_data['metrics_history']
        eval_metrics = metrics_history['eval']
        epochs = metrics_history['epoch']
        
        f1_scores = [metrics['f1'] for metrics in eval_metrics]
        best_f1_idx = np.argmax(f1_scores)
        best_f1 = f1_scores[best_f1_idx]
        best_epoch = epochs[best_f1_idx]
        
        best_epochs_data[trial_num] = {
            'best_epoch': best_epoch,
            'best_f1': best_f1,
            'learning_rate': trial_data['hyperparameters']['learning_rate'],
            'best_metrics': eval_metrics[best_f1_idx]
        }
    
    return best_epochs_data
best_epochs_data = find_best_epochs_metrics(hyperparameter_dict)

In [5]:
num_epochs = best_epochs_data['18']['best_epoch']

In [6]:
dropout = study.best_params["dropout"]
learning_rate = study.best_params["learning_rate"]
weight_decay = study.best_params["weight_decay"]
batch_size = 2

In [7]:
print('num epochs:', num_epochs)
print('dropout:', dropout)
print('learning rate:', learning_rate)
print('weight decay:', weight_decay)
print('batch size:', batch_size)

num epochs: 11.0
dropout: 0.3560592494053517
learning rate: 3.127610016621582e-05
weight decay: 0.000817139913231531
batch size: 2


In [25]:
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import default_data_collator
from transformers.trainer_callback import TrainerCallback
import os
import json
import numpy as np


class MetricsTrackingCallback(TrainerCallback):
    def __init__(self):
        self.metrics_history = {
            'train': [],
            'eval': [],
            'epoch': []
        }
        self.current_train_metrics = None
        self.last_logged_epoch = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Called after evaluation"""
        if metrics and state.epoch != self.last_logged_epoch:
            # Store evaluation metrics
            eval_metrics = {
                'precision': metrics.get('eval_precision', None),
                'recall': metrics.get('eval_recall', None),
                'f1': metrics.get('eval_f1', None),
                'accuracy': metrics.get('eval_accuracy', None),
                'loss': metrics.get('eval_loss', None)
            }
            
            # Store training metrics that were saved during training
            train_metrics = {}
            if self.current_train_metrics is not None:
                train_metrics = self.current_train_metrics
                self.current_train_metrics = None  # Reset for next epoch
            
            # Append metrics for this epoch
            self.metrics_history['train'].append(train_metrics)
            self.metrics_history['eval'].append(eval_metrics)
            self.metrics_history['epoch'].append(state.epoch)
            self.last_logged_epoch = state.epoch
            
            # Save to file
            save_dir = args.output_dir
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, 'metrics_history.json'), 'w') as f:
                json.dump(self.metrics_history, f, indent=2)

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Called on each training log"""
        if logs:
            # Check if these are training metrics (not eval metrics)
            if all(not k.startswith('eval_') for k in logs.keys()):
                # Store the most recent training metrics
                self.current_train_metrics = {
                    'loss': logs.get('loss', None),
                    'learning_rate': logs.get('learning_rate', None)
                }


In [26]:
train_dict = {}
for i in range(3):
    
    model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                             id2label=label_map,
                                                             label2id=label_map_reversed).to(device)
    
    model.config.hidden_dropout_prob = dropout
    model.config.attention_probs_dropout_prob = dropout

    metrics_callback = MetricsTrackingCallback()


    trial_output_dir = f'./results_ner_boxes/{i}'
    training_args = TrainingArguments(
        output_dir=trial_output_dir,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,  
        weight_decay=weight_decay,
        logging_steps=10,
        evaluation_strategy="epoch",
        metric_for_best_model="eval_f1",
        load_best_model_at_end=True,
        save_strategy="epoch",
        save_total_limit=1,
        
    ) 

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=processor,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks=[metrics_callback]
    )

    train_result = trainer.train()

    eval_results = trainer.evaluate()

    train_dict[i] = {
    "final_eval_results": eval_results,
    "metrics_history": metrics_callback.metrics_history,
}


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/825 [00:00<?, ?it/s]c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
  1%|▏         | 11/825 [00:02<02:35,  5.23it/s]

{'loss': 1.7473, 'grad_norm': 6.216126441955566, 'learning_rate': 3.089699592177684e-05, 'epoch': 0.13}


  3%|▎         | 21/825 [00:04<02:28,  5.41it/s]

{'loss': 1.4874, 'grad_norm': 10.920066833496094, 'learning_rate': 3.051789167733786e-05, 'epoch': 0.27}


  4%|▍         | 31/825 [00:06<02:28,  5.35it/s]

...

100%|██████████| 825/825 [03:32<00:00,  5.91it/s]c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(

100%|██████████| 825/825 [03:38<00:00,  5.91it/s]

{'eval_loss': 0.7498010396957397, 'eval_precision': 0.8072289156626506, 'eval_recall': 0.8320914058618977, 'eval_f1': 0.8194716242661448, 'eval_accuracy': 0.8253892784975633, 'eval_runtime': 1.7889, 'eval_samples_per_second': 27.949, 'eval_steps_per_second': 13.975, 'epoch': 11.0}


100%|██████████| 825/825 [03:42<00:00,  3.71it/s]
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'train_runtime': 222.1989, 'train_samples_per_second': 7.376, 'train_steps_per_second': 3.713, 'train_loss': 0.36046468420462174, 'epoch': 11.0}


100%|██████████| 25/25 [00:01<00:00, 14.30it/s]


In [28]:
with open("3ser_lm3_ner_boxes_hptune_models_results.json", "w") as json_file:
    json.dump(train_dict, json_file)